In [1]:
import geoglows
import netCDF4 as nc
import plotly
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import datetime
import calendar

In [2]:
fr = nc.Dataset('forecast_record-2020-central_america-geoglows.nc')

In [3]:
time = pd.to_datetime(fr['time'][:].data, unit='s', origin='unix')
flow = list(fr['Qout'][list(fr['rivid'][:]).index(950212), :].data)

In [4]:
df = pd.DataFrame(np.transpose(np.asarray([time,flow])), columns=['time', 'flow'])
df[df['flow'] > 10000] = np.nan
df = df.dropna()

In [5]:
#df.to_csv('check.csv')
df

,time,flow
736,2020-04-02 00:00:00,15.3169
737,2020-04-02 03:00:00,15.2415
738,2020-04-02 06:00:00,15.0409
739,2020-04-02 09:00:00,14.8168
740,2020-04-02 12:00:00,14.6387
...,...,...
843,2020-04-15 09:00:00,15.3023
844,2020-04-15 12:00:00,14.9848
845,2020-04-15 15:00:00,14.6957
846,2020-04-15 18:00:00,14.4182


In [6]:
stats = geoglows.streamflow.forecast_stats(950212)

In [7]:
stats

,datetime,high_res (m3/s),mean (m3/s),std_dev_range_upper (m3/s),std_dev_range_lower (m3/s),min (m3/s),max (m3/s)
0,2020-04-15 00:00:00,16.203131,16.203131,16.203131,16.203131,16.203131,16.203131
1,2020-04-15 01:00:00,16.157442,NaN,NaN,NaN,NaN,NaN
2,2020-04-15 02:00:00,16.054276,NaN,NaN,NaN,NaN,NaN
3,2020-04-15 03:00:00,15.946671,16.045527,16.059469,16.031584,15.946671,16.052742
4,2020-04-15 04:00:00,15.807544,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
120,2020-04-24 00:00:00,15.374204,28.649071,44.337723,12.960419,10.870433,97.350075
121,2020-04-24 06:00:00,15.529033,28.495476,44.357159,12.633793,11.288983,85.627678
122,2020-04-24 12:00:00,15.565354,28.491133,45.530930,11.451336,11.499772,97.397789
123,2020-04-24 18:00:00,15.304013,27.623936,45.290150,9.957722,11.157758,105.569557


In [12]:
# Start processing the inputs
dates = stats['datetime'].tolist()
startdate = dates[0]
enddate = dates[-1]

plot_data = {
#     'reach_id': reach_id,
#     'drain_area': drain_area,
    'x_ensembles': stats[['datetime', 'mean (m3/s)']].dropna(axis=0)['datetime'].tolist(),
    'x_hires': stats[['datetime', 'high_res (m3/s)']].dropna(axis=0)['datetime'].tolist(),
    'y_max': max(stats['max (m3/s)']),
    'min': list(stats['min (m3/s)'].dropna(axis=0)),
    'mean': list(stats['mean (m3/s)'].dropna(axis=0)),
    'max': list(stats['max (m3/s)'].dropna(axis=0)),
    'stdlow': list(stats['std_dev_range_lower (m3/s)'].dropna(axis=0)),
    'stdup': list(stats['std_dev_range_upper (m3/s)'].dropna(axis=0)),
    'hires': list(stats['high_res (m3/s)'].dropna(axis=0)),
#     'r2': rperiods.iloc[3][0],
#     'r10': rperiods.iloc[2][0],
#     'r20': rperiods.iloc[1][0],
}

meanplot = go.Scatter(
    name='Mean',
    x=plot_data['x_ensembles'],
    y=plot_data['mean'],
    line=dict(color='blue'),
)
maxplot = go.Scatter(
    name='Max',
    x=plot_data['x_ensembles'],
    y=plot_data['max'],
    fill='tonexty',
    mode='lines',
    line=dict(color='rgb(152, 251, 152)', width=0)
)
minplot = go.Scatter(
    name='Min',
    x=plot_data['x_ensembles'],
    y=plot_data['min'],
    fill=None,
    mode='lines',
    line=dict(color='rgb(152, 251, 152)')
)
stdlowplot = go.Scatter(
    name='Std. Dev. Lower',
    x=plot_data['x_ensembles'],
    y=plot_data['stdlow'],
    fill='tonexty',
    mode='lines',
    line=dict(color='rgb(152, 251, 152)', width=0)
)
stdupplot = go.Scatter(
    name='Std. Dev. Upper',
    x=plot_data['x_ensembles'],
    y=plot_data['stdup'],
    fill='tonexty',
    mode='lines',
    line={'width': 0, 'color': 'rgb(34, 139, 34)'}
)
hires = go.Scatter(
    name='Higher Resolution',
    x=plot_data['x_hires'],
    y=plot_data['hires'],
    line={'color': 'black'}
)
recordplot = go.Scatter(
    name='Forecast Record',
    x=list(df['time']),
    y=list(df['flow']),
    line={'color': 'purple'}
)
layout = go.Layout(
    title="Sample Hydrograph with Forecast Records",
    xaxis={'title': 'Date'},
    yaxis={
        'title': 'Streamflow (m<sup>3</sup>/s)',
        'range': [0, 1.2 * plot_data['y_max']]
    },
)
figure = go.Figure([minplot, stdlowplot, stdupplot, maxplot, meanplot, hires, recordplot], layout=layout)

In [14]:
figure.show()